In [58]:
from google.cloud import aiplatform
from google.oauth2 import service_account

# 1. Path to your key
KEY_PATH = "/Users/dlangford/Desktop/gcp/gen-lang-client-0085631554-8408cebe1fe7.json"

# 2. Create the credentials object
creds = service_account.Credentials.from_service_account_file(KEY_PATH)

In [60]:
# 1. Initialize Vertex AI
# Replace with your project ID
PROJECT_ID = "gen-lang-client-0085631554" 
REGION = "us-central1"
# us-central1-docker.pkg.dev/gen-lang-client-0085631554/simulations/covid-jax-tpu:latest
CONTAINER_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/simulations/covid-jax-tpu:latest"

aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    credentials=creds,
    staging_bucket="gs://tpu-jax-bucket",
)

job = aiplatform.CustomJob(
    display_name="covid-jax-tpu-simulation-v2",
    worker_pool_specs=[
        {
            "replica_count": 1,
            "machine_spec": {
                "machine_type": "ct5lp-hightpu-1t",
                "tpu_topology": "1x1",
            },
            "container_spec": {
                "image_uri": CONTAINER_URI,
                #"args": ["--config_file", "reference/novel-respiratory-advanced-example-config.json"],
            },
        }
    ],
)

job.run(sync=False)
print("Submitted:", job.resource_name)

# 3. Create the Custom Job
# This 'packages' your container into a job definition
#job = aiplatform.CustomContainerTrainingJob(
#    display_name="covid-jax-tpu-simulation",
#    container_uri=CONTAINER_URI,
#)

# 4. Run the Job on a TPU v5e
# 'ct5lp-hightpu-1t' is the machine type for a TPU v5e (1 chip)
# accelerator_type 'TPU_V5E' tells Vertex AI to attach the hardware
# args passes your config path to your script's argparse
# model_job = job.run(
#     args=["--config_file", "reference/novel-respiratory-advanced-example-config.json"],
#     replica_count=1,
#     machine_type="ct5lp-hightpu-1t",
#     accelerator_type="TPU_V5_LITEPOD",
#     accelerator_count=1,
#     tpu_topology="1x1",
#     sync=False # Setting to False lets your notebook keep running while the job starts
# )

# print(f"Job submitted! Monitor it here: {model_job.resource_name}")

Creating CustomJob


RuntimeError: CustomJob resource has not been created.

In [43]:
from google.cloud import storage

# Initialize client
client = storage.Client(credentials=creds)
bucket = client.bucket("tpu-jax-bucket")

# List all files in that specific job directory
prefix = "aiplatform-custom-training-2026-01-12-15:38:06.011/"
blobs = bucket.list_blobs(prefix=prefix)

for blob in blobs:
    print(f"Found file: {blob.name}")
    # To download a specific file:
    # blob.download_to_filename(blob.name.split('/')[-1])

In [55]:
# This lists the 5 most recent jobs in your region
recent_jobs = aiplatform.CustomJob.list(order_by="create_time desc")
for j in recent_jobs:
    print(f"Job Name: {j.display_name} | Status: {j.state}")

Job Name: covid-jax-tpu-simulation | Status: 2
Job Name: covid-jax-non-tpu-v5-custom-job | Status: 4
Job Name: covid-jax-non-tpu-v4-custom-job | Status: 5
Job Name: covid-jax-non-tpu-v3-custom-job | Status: 4
Job Name: covid-jax-non-tpu-v2-custom-job | Status: 4
Job Name: covid-jax-non-tpu-custom-job | Status: 4
Job Name: covid-jax-tpu-simulation | Status: 5
Job Name: covid-jax-tpu-simulation | Status: 7


In [50]:
job

resource name: projects/640753469186/locations/us-central1/customJobs/8417369729534001152